# Assignment 5

In this assignment, your task is to scrape data from the given [link](https://resultadoshistorico.onpe.gob.pe/EG2021/). You'll need to extract all votes for departments, provinces, and districts. Follow these steps:

1. Click on 'Resumen General'.
2. Select 'Presidencial', and then choose 'Peru'.
3. Scrape the data and generate a final table with the following columns: department, province, district, political_organization, total_votes.

Your code must execute successfully; any errors will result in a zero grade. Ensure you test your code thoroughly.

In [155]:
 #1
    
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [173]:
#installing libraries

!pip install selenium
!pip install webdriver-manager
!pip install lxml
!pip install lxmunidecodel

ERROR: Could not find a version that satisfies the requirement lxmunidecodel (from versions: none)
ERROR: No matching distribution found for lxmunidecodel


In [174]:
pwd

'C:\\Users\\Marco\\GitHub\\MediaLab_Summer_Python\\assignments\\assignment5'

In [196]:
from selenium import webdriver

from webdriver_manager.chrome import ChromeDriverManager
import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import unidecode
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service

In [197]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service

# service = Service(executable_path=r"C:\Users\Marco\GitHub\MediaLab_Summer_Python\Lecture_7\chromedriver-win64\chromedriver.exe")
# driver = webdriver.Chrome(service=service)

In [198]:
service = Service(executable_path=r"C:\Users\Marco\GitHub\MediaLab_Summer_Python\Lecture_7\chromedriver-win64\chromedriver.exe")
driver = webdriver.Chrome(service=service)

url_1 = "https://resultadoshistorico.onpe.gob.pe/EG2021/"

driver.get( url_1 )
driver.maximize_window()

## 1. Click on 'Resumen General'

In [199]:
driver.find_element(By.XPATH, '/html/body/onpe-root/onpe-home-onpe/div[1]/div/div/div/div[2]/div[1]/div/div/a/div[2]/div').click()
time.sleep(2)

## 2. Select 'Presidencial', and then choose 'Peru'

In [200]:
#Clicking automation by using time.sleep function
driver.find_element(By.XPATH, '/html/body/onpe-root/onpe-layout-container/onpe-menu/div/nav/div/div/div[2]/div/div[2]/a').click()
time.sleep(3)
driver.find_element(By.XPATH, '/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div/select/option[2]').click()

## 3. Scrape the data and generate a final table with the following columns: department, province, district, political_organization, total_votes.

In [201]:
# Store all_tables
all_tables = {}

# Departments - create a loop for departments
for dept_idx in range(26):
    try:
        # Get all elements 
        dept = Select(driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select"))
        department = dept.options[dept_idx]
        
        # Get department names
        dept_name = department.text
        
        # Select a different department name
        if dept_name != "--TODOS--":
            department.click()

            # Provinces
            provi = Select(driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select"))
            num_provi_options = len(provi.options)

            # Create a loop for provinces
            for provi_idx in range(num_provi_options):
                try:
                    # Get all elements 
                    provi = Select(driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select"))
                    province = provi.options[provi_idx]

                    provi_name = province.text

                    if provi_name != "--TODOS--":
                        # Click on province
                        province.click()

                        # Districts
                        distri = Select(driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select"))
                        num_distri_options = len(distri.options)

                        # Create a loop for districts
                        for distri_idx in range(num_distri_options):
                            try:
                                # Get all elements
                                distri = Select(driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select"))
                                district = distri.options[distri_idx]

                                # Get district names
                                distri_name = district.text

                                # We select a different district 
                                if distri_name != "-- SELECCIONE --":
                                    district.click()

                                    # Get UBIGEO
                                    ubigeo = driver.current_url.split("/")[-1]

                                    # Get table of presidential votes
                                    table_path = driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[4]/div[1]/div[3]/div")
                                    table_html = table_path.get_attribute('innerHTML')
                                    # Use pandas
                                    table = pd.read_html(table_html)

                                    # Clean
                                    row_new_columns = table[0].iloc[0, 2:]
                                    clean_columns = row_new_columns \
                                                      .str.replace(" ", "_") \
                                                      .str.lower().str.replace("%", "share_") \
                                                      .apply(lambda x: unidecode.unidecode(x)) \
                                                      .tolist()

                                    # Selecting specific columns
                                    table_clean = table[0].iloc[1:, 2:].copy()

                                    table_clean.columns = clean_columns

                                    # New values to columns 
                                    table_clean['department'] = dept_name
                                    table_clean['province'] = provi_name
                                    table_clean['district'] = distri_name
                                    table_clean[ 'ubigeo' ] = ubigeo


                                    # Store tables
                                    all_tables[ubigeo] = table_clean
                            except:
                                print(f"there is an error in region {dept_idx}")
                                pass
                except:
                    print(f"there is an error in region {dept_idx}")
                    pass
    except:
        print(f"there is an error in region {dept_idx}")
        pass

In [202]:
final_result = pd.concat( all_tables.values() ).reset_index( drop = True )
final_result

,organizaciones_politicas,total,share_validos,share_emitidos,department,province,district,ubigeo
0,PARTIDO NACIONALISTA PERUANO,356,9.327%,6.389%,AMAZONAS,BAGUA,ARAMANGO,010202
1,"EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD",13,0.341%,0.233%,AMAZONAS,BAGUA,ARAMANGO,010202
2,PARTIDO MORADO,41,1.074%,0.736%,AMAZONAS,BAGUA,ARAMANGO,010202
3,PERU PATRIA SEGURA,4,0.105%,0.072%,AMAZONAS,BAGUA,ARAMANGO,010202
4,VICTORIA NACIONAL,151,3.956%,2.710%,AMAZONAS,BAGUA,ARAMANGO,010202
...,...,...,...,...,...,...,...,...
33985,ALIANZA PARA EL PROGRESO,292,35.394%,20.434%,UCAYALI,PURUS,PURUS,250401
33986,TOTAL DE VOTOS VÁLIDOS,825,100.000%,57.733%,UCAYALI,PURUS,PURUS,250401
33987,VOTOS EN BLANCO,422,NaN,29.531%,UCAYALI,PURUS,PURUS,250401
33988,VOTOS NULOS,182,NaN,12.736%,UCAYALI,PURUS,PURUS,250401
